In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Masking
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pickle

# Define the dataset (input sequences and corresponding sentences)
signs = [
    ["Hello"],
    ["Hello", "How", "You"],  # "Hello, how are you?"
    ["How", "You"],  # "How are you?"
    ["I Love You"],  # "I love you."
    ["Thank", "You"],  # "Thank you!"
    ["Hello", "Yes"],  # "Hello, yes!"
    ["Iam", "Want", "Water"],  # "I want water."
    ["Iam", "Want", "Eat"],  # "I want to eat."
    ["You", "Eat"],  # "Are you eating?"
    ["You", "Sleep"],  # "Are you sleeping?"
    ["Iam", "Sleep"],  # "I am sleeping."
    ["Want", "Water"],  # "I need water."
    ["Want", "Eat"],  # "I need food."
    ["Iam", "Ok"],  # "I am okay."
    ["Ok", "Thank", "You"],  # "Okay, thank you!"
    ["Hello", "Iam"],  # "Hello, I am here."
    ["How", "Iam"],  # "How am I?"
    ["Want", "Sleep"],  # "I want to sleep."
    ["Beat", "You"],  # "I will beat you!"
    ["Hello", "Thank", "You"],  # "Hello, thank you."
    ["You", "Want", "Water"],  # "Do you want water?"
]

# Corresponding sentences
sentences = [
    "Hello",
    "Hello, how are you?",
    "How are you?",
    "I love you.",
    "Thank you!",
    "Hello, yes!",
    "I want water.",
    "I want to eat.",
    "Are you eating?",
    "Are you sleeping?",
    "I am sleeping.",
    "I need water.",
    "I need food.",
    "I am okay.",
    "Okay, thank you!",
    "Hello, I am here.",
    "How am I?",
    "I want to sleep.",
    "I will beat you!",
    "Hello, thank you.",
    "Do you want water?",
]

# Initialize Tokenizer and fit on the sign words
input_tokenizer = Tokenizer(oov_token="<OOV>")
input_tokenizer.fit_on_texts(signs)

# Encode the input sequences
input_sequences = input_tokenizer.texts_to_sequences(signs)
max_input_length = max(len(seq) for seq in input_sequences)
input_sequences_padded = pad_sequences(input_sequences, maxlen=max_input_length, padding='post')

# Prepare the output tokenizer and encode sentences
output_tokenizer = Tokenizer(oov_token="<OOV>")
output_tokenizer.fit_on_texts(sentences)
output_sequences = output_tokenizer.texts_to_sequences(sentences)
max_output_length = max(len(seq) for seq in output_sequences)
output_sequences_padded = pad_sequences(output_sequences, maxlen=max_output_length, padding='post')

# Convert output sequences to categorical
vocab_size_output = len(output_tokenizer.word_index) + 1
output_sequences_categorical = np.array([to_categorical(seq, num_classes=vocab_size_output) for seq in output_sequences_padded])

# Define the model architecture
embedding_dim = 50  # Dimension of the embedding space
latent_dim = 64  # Number of units in the LSTM

model = Sequential()
model.add(Embedding(input_dim=len(input_tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_input_length, mask_zero=True))
# Change return_sequences to False to output a single hidden state vector
model.add(LSTM(latent_dim))
# Add a RepeatVector layer to repeat the hidden state vector to match the target sequence length
model.add(keras.layers.RepeatVector(max_output_length))
model.add(LSTM(latent_dim, return_sequences=True))
model.add(Dense(vocab_size_output, activation='softmax'))


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(input_sequences_padded, output_sequences_categorical, epochs=300, batch_size=16, verbose=1)

# Save the model and tokenizers
model.save('sentence_formation_model.h5')
with open('input_tokenizer.pkl', 'wb') as f:
    pickle.dump(input_tokenizer, f)
with open('output_tokenizer.pkl', 'wb') as f:
    pickle.dump(output_tokenizer, f)
with open('max_seq_lengths.pkl', 'wb') as f:
    pickle.dump({
        'max_encoder_seq_length': max_input_length,
        'max_decoder_seq_length': max_output_length,
        'latent_dim': latent_dim
    }, f)

print("Model and tokenizers saved successfully.")

Epoch 1/300
2/2 [==============================] - 4s 9ms/step - loss: 3.2179 - accuracy: 0.0595
Epoch 2/300
2/2 [==============================] - 0s 8ms/step - loss: 3.2121 - accuracy: 0.2262
Epoch 3/300
2/2 [==============================] - 0s 8ms/step - loss: 3.2057 - accuracy: 0.2262
Epoch 4/300
2/2 [==============================] - 0s 8ms/step - loss: 3.1989 - accuracy: 0.2262
Epoch 5/300
2/2 [==============================] - 0s 7ms/step - loss: 3.1910 - accuracy: 0.2262
Epoch 6/300
2/2 [==============================] - 0s 6ms/step - loss: 3.1817 - accuracy: 0.2262
Epoch 7/300
2/2 [==============================] - 0s 6ms/step - loss: 3.1703 - accuracy: 0.2262
Epoch 8/300
2/2 [==============================] - 0s 6ms/step - loss: 3.1558 - accuracy: 0.2262
Epoch 9/300
2/2 [==============================] - 0s 7ms/step - loss: 3.1391 - accuracy: 0.2262
Epoch 10/300
2/2 [==============================] - 0s 7ms/step - loss: 3.1168 - accuracy: 0.2262
Epoch 11/300
2/2 [===========

c:\Users\VENKATA REDDY\envs\testold\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Load the trained model and tokenizers
model = load_model('sentence_formation_model.h5')

with open('input_tokenizer.pkl', 'rb') as f:
    input_tokenizer = pickle.load(f)

with open('output_tokenizer.pkl', 'rb') as f:
    output_tokenizer = pickle.load(f)

with open('max_seq_lengths.pkl', 'rb') as f:
    seq_lengths = pickle.load(f)
    max_encoder_seq_length = seq_lengths['max_encoder_seq_length']
    max_decoder_seq_length = seq_lengths['max_decoder_seq_length']

# Reverse the output tokenizer for decoding
reverse_output_word_index = {v: k for k, v in output_tokenizer.word_index.items()}

# Function to add punctuation based on sentence structure
def add_punctuation(sentence):
    if sentence.startswith("how") or sentence.startswith("are") or sentence.startswith("do"):
        return sentence.capitalize() + "?"
    elif sentence.startswith("hello"):
        return sentence.capitalize() + ","
    elif sentence.endswith("you") or sentence.endswith("water") or sentence.endswith("eat") or sentence.endswith("sleep"):
        return sentence.capitalize() + "."
    else:
        return sentence.capitalize()

# Function to predict a sentence based on input signs
def predict_sentence(sign_sequence):
    # Convert sign sequence to integers using the input tokenizer
    input_sequence = input_tokenizer.texts_to_sequences([sign_sequence])
    input_sequence_padded = pad_sequences(input_sequence, maxlen=max_encoder_seq_length, padding='post')

    # Predict the output sequence
    predictions = model.predict(input_sequence_padded)
    output_sequence = np.argmax(predictions, axis=-1)

    # Decode the output sequence to words
    decoded_sentence = []
    for idx in output_sequence[0]:
        if idx == 0:
            break
        word = reverse_output_word_index.get(idx, '')
        decoded_sentence.append(word)

    # Join words and add punctuation
    sentence = ' '.join(decoded_sentence)
    sentence_with_punctuation = add_punctuation(sentence)
    return sentence_with_punctuation

# Test cases (input as list of signs)
test_cases = [
    ["Hello", "How", "You"],         # "Hello, how are you?"
    ["Iam", "Want", "Water"],        # "I want water."
    ["I", "Love", "You"],            # "I love you."
    ["You", "Sleep"],                # "Are you sleeping?"
    ["Want", "Sleep"],               # "I want to sleep."
    ["Thank", "You"],                # "Thank you!"
    ["Hello", "Iam"],                # "Hello, I am here."
    ["Beat", "You"],                 # "I will beat you!"
    ["Ok", "Thank", "You"],          # "Okay, thank you!"
    ["You", "Want", "Water"]         # "Do you want water?"
]

# Predict and display results for each test case
for signs in test_cases:
    predicted_sentence = predict_sentence(signs)
    print(f"Signs: {signs} -> Sentence: {predicted_sentence}")

1/1 [==============================] - 1s 898ms/step
Signs: ['Hello', 'How', 'You'] -> Sentence: Hello how are you,
1/1 [==============================] - 0s 17ms/step
Signs: ['Iam', 'Want', 'Water'] -> Sentence: I want water.
1/1 [==============================] - 0s 15ms/step
Signs: ['I', 'Love', 'You'] -> Sentence: I you.
1/1 [==============================] - 0s 17ms/step
Signs: ['You', 'Sleep'] -> Sentence: Are you sleeping?
1/1 [==============================] - 0s 17ms/step
Signs: ['Want', 'Sleep'] -> Sentence: I want to sleep.
1/1 [==============================] - 0s 16ms/step
Signs: ['Thank', 'You'] -> Sentence: Thank you.
1/1 [==============================] - 0s 14ms/step
Signs: ['Hello', 'Iam'] -> Sentence: Hello i am here,
1/1 [==============================] - 0s 15ms/step
Signs: ['Beat', 'You'] -> Sentence: I will beat you.
1/1 [==============================] - 0s 16ms/step
Signs: ['Ok', 'Thank', 'You'] -> Sentence: Okay thank you.
1/1 [==============================] 